<a href="https://colab.research.google.com/github/marco-siino/SemEval2024/blob/main/Task%209/SemEval2024_Task9_DeBERTav3_SUBMITTED_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install utils

import os
import json
import sys
import torch
import random
import numpy as np
import random
random.seed(10)
from tqdm import tqdm
from utils import *
from transformers import pipeline
import logging
import argparse
device="cuda:0"
%load_ext autoreload
%autoreload 2

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=c22a68c56233d403fd34ce5d76f0edea5a8cf2efc0ec4100657f6f0c2240f78d
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [ ]:
def getResultdata(result_data):
    choice_to_index = {'A':0,'B':1,'C':2,'D':3}

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1

    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)

    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    for item in word_play.values():
        word_data_list.append(item)
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)

    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')
    final_result['sentence'] = getMetric(reverse_data_list)


    all_data = word_data_list + reverse_data_list
    print('#########All data##########')
    final_result['all'] = getMetric(all_data)

    return final_result

In [ ]:
# Evaluation ds pre-competition.
eval_sentence_data_path = "SP_eval_data_for_practice.npy"
eval_wordplay_data_list = "WP_eval_data_for_practice.npy"
eval_sentence_data_list = list(np.load(eval_sentence_data_path,allow_pickle=True))
eval_wordplay_data_list = list(np.load(eval_wordplay_data_list,allow_pickle=True))


# Train set labelled.
#train_sentence_data_path = "SP-train.npy"
#train_wordplay_data_list = "WP-train.npy"
#train_sentence_data_list = list(np.load(train_sentence_data_path,allow_pickle=True))
#train_wordplay_data_list = list(np.load(train_wordplay_data_list,allow_pickle=True))

# Official test set.
test_sentence_data_path = "SP_new_test.npy"
test_wordplay_data_list = "WP_new_test.npy"
test_sentence_data_list = list(np.load(test_sentence_data_path,allow_pickle=True))
test_wordplay_data_list = list(np.load(test_wordplay_data_list,allow_pickle=True))


In [ ]:
few_shot_examples = train_sentence_data_list

NameError: name 'train_sentence_data_list' is not defined

In [ ]:
test_data_list = test_sentence_data_list + test_wordplay_data_list
len(test_data_list)

216

# Few-shot setup

In [ ]:
def get_fewshot_demo(examples,number):
    label_to_answer = ['A','B','C','D']
    few_shot_demon = examples[:number//2]+examples[4:4+number//2]
    fewshot = ''
    for sample in few_shot_demon:
        fewshot += \
"""
Question: {}

What is the correct answer to the question from the following choices?
(A): {}
(B): {}
(C): {}
(D): {}

Answer: ({})
""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3],label_to_answer[int(sample['label'])])

    return fewshot

def get_single_demo(sample):
    sample_demo = \
"""
Question: {}

What is the correct answer to the question from the following choices?
Options:
(A): {}
(B): {}
(C): {}
(D): {}""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])

    return sample_demo

In [ ]:
### NUOVA VERSIONE CHE COSTRUISCE LE FRASI IN MANIERA OPPORTUNISTICA.

def get_fewshot_demo(examples,number):
    label_to_answer = ['A','B','C','D']
    few_shot_demon = examples[:number//2]+examples[4:4+number//2]
    fewshot = ''
    for sample in few_shot_demon:
        fewshot += \
"""
Question: {}
Which one of the following is the most simple and reasonable explanation to the previous question? (Be smart it is a brain teaser, try to avoid creative answers)
(A): {}
(B): {}
(C): {}
(D): {}

Answer: ({})
""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3],
           label_to_answer[int(sample['label'])])

    return fewshot

def get_single_demo(sample):
    sample_demo = \
"""

Question: {}
Which one of the following is the most simple and reasonable explanation to the previous question? (Be smart it is a brain teaser, try to avoid creative answers)
(A): {}
(B): {}
(C): {}
(D): {}
""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])
    return sample_demo

In [ ]:
demon_num = 10
demonstration = get_fewshot_demo(few_shot_examples,demon_num)

In [ ]:
print(demonstration)

In [ ]:
print(get_single_demo(test_sentence_data_list[2]))


Question: How many years in your life it happens that before your birthday you'd be one year younger than tomorrow

What is the correct answer to the question from the following choices?
Options:
(A): In ech leap year.
(B): In the first year of graduation.
(C): It happens every year.
(D): None of above.


In [ ]:
qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")

for sample in tqdm(test_sentence_data_list):

  question = sample['question'] + "  \n What is the correct answer to the brain teaser question from the following choices? (Pick only one Option (A)-(D)"

  context = """



  Options:

  (A): {}

  (B): {}

  (C): {}

  (D): {}

  """.format(sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])

  print("\n")
  response=qa_model(question = question, context = context)

  print("\nQuestion:",question)
  print(context)

  print("Model's answer is:",response['answer'])

  for i in range(0,4):
    if response['answer'].strip() in sample['choice_list'][i]:
      sample['response']=i
      print("The answer is:",i)
      with open('answer_sen.txt', 'a') as f:
        f.write(str(i))
        f.write('\n')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

...


Question: He has wed numerous women, but never himself. He is who?  
 What is the correct answer to the brain teaser question from the following choices? (Pick only one Option (A)-(D)




  Options:

  (A): A teacher.

  (B): A preacher.

  (C): A laywer.

  (D): None of above.

  
Model's answer is:  A teacher.
The answer is: 0




 99%|█████████▉| 119/120 [01:49<00:00,  1.21it/s]


Question: You walk into a room and see a bed and lie on it. On the bed, there are two dogs, five cats, a giraffe, six cows, and a goose. There are also three doves flying above the bed. How many legs are on the floor?  
 What is the correct answer to the brain teaser question from the following choices? (Pick only one Option (A)-(D)




  Options:

  (A): Thirty-six, as there are eighteen animals.

  (B): Six. The bed's four legs, plus your two legs.

  (C): Seventy-two, as there are eighteen animals.

  (D): None of above.

  
Model's answer is:  Thirty-six,
The answer is: 0




100%|██████████| 120/120 [01:50<00:00,  1.08it/s]


Question: Four of Mrs. Jones' daughters exist. She has one brother for each of her daughters. Mrs. Jones, however, only has five kids. How is it even doable?  
 What is the correct answer to the brain teaser question from the following choices? (Pick only one Option (A)-(D)




  Options:

  (A): Some brothers were not loved by family and moved away.

  (B): Some daughters get married and have their own family.

  (C): Each daughter shares the same brother.

  (D): None of above.

  
Model's answer is:  None of above.
The answer is: 3


In [ ]:
# Before running this cell add the answer to the question 45.
!zip -r 'submission.zip' 'answer_sen.txt'

  adding: answer_sen.txt (deflated 69%)


In [ ]:
qa_model = pipeline("question-answering", "EleutherAI/gpt-neo-1.3B").to_device

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like EleutherAI/gpt-neo-1.3B is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
print(device)

cuda:0


In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

2024-01-24 13:19:45.539820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 62403 MB memory:  -> device: 0, name: NVIDIA PG506-243, pci bus id: 0000:1d:00.0, compute capability: 8.0
2024-01-24 13:19:45.543084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 62403 MB memory:  -> device: 1, name: NVIDIA PG506-243, pci bus id: 0000:56:00.0, compute capability: 8.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 349889915592370980
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 65434943488
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 7312045900287581865
 physical_device_desc: "device: 0, name: NVIDIA PG506-243, pci bus id: 0000:1d:00.0, compute capability: 8.0"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 65434943488
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 14553817067647267052
 physical_device_desc: "device: 1, name: NVIDIA PG506-243, pci bus id: 0000:56:00.0, compute capability: 8.0"
 xla_global_id: 2144165316]

In [ ]:
qa_model = pipeline("question-answering", "EleutherAI/gpt-neox-20b", device_map="auto")

Loading checkpoint shards: 100%|████████████████| 46/46 [02:58<00:00,  3.87s/it]
Some weights of GPTNeoXForQuestionAnswering were not initialized from the model checkpoint at EleutherAI/gpt-neox-20b and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast

model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b", device_map="auto")
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/gpt-neox-20b", device_map="auto")

Loading checkpoint shards: 100%|████████████████| 46/46 [03:01<00:00,  3.94s/it]


In [ ]:
def getFewshot_demo(examples,number):
    label_to_answer = ['A','B','C','D']
    few_shot_demon = examples[:number//2]+examples[4:4+number//2]
    fewshot = ''
    for sample in few_shot_demon:
        fewshot += """

Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:({}) {}""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1]\
                    ,sample['choice_list'][2],sample['choice_list'][3]\
                         ,label_to_answer[int(sample['label'])],sample['choice_list'][int(sample['label'])])

    return fewshot

In [ ]:
def getSample_demo(sample):
    sample_demo = """

Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])

    return sample_demo

In [ ]:
demonstration = """Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"""
demon_num = 10
demonstration = demonstration + getFewshot_demo(train_sentence_data_list,demon_num)

In [ ]:
i=0
for sample in tqdm(test_sentence_data_list):
  i+=1

  print("\n\n - - - - SAMPLE ",i," - - - -" )

  question =  "  \n\n Please pick the best choice for the brain teaser question below. Each brain teaser has only one possible solution including the choice none of above, answer should only provide one choice."""


  context =  """

  Question: {}

  Choice:
  (A) {}
  (B) {}
  (C) {}
  (D) {}

  """.format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])

  prompt = getSample_demo(sample)

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids

  gen_tokens = model.generate(
    input_ids,
    temperature = 0.1,
    top_p = 0.0,
    max_length=1400
  )
  gen_text = tokenizer.batch_decode(gen_tokens)[0]

  print("Answer is: ",gen_text, "\n\n")

  #for i in range(0,4):
    #if response['answer'].strip() in sample['choice_list'][i]:
      #sample['response']=i
      #print("The answer is:",i)



  0%|                                                   | 0/120 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




 - - - - SAMPLE  1  - - - -


In [ ]:
for sample in tqdm(test_sentence_data_list):

  question =  "  \n Please pick the best option for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"""
  question += sample['question']

  context =  """

  Options:

  (A): {}

  (B): {}

  (C): {}

  (D): {}

  """.format(sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])

  print("\n")

  input_q = question+context
  response=qa_model(question = question, context = context)

  print("\nQuestion:",question)
  print(context)

  print("l'oggetto response è:",response['answer'])

  for i in range(0,4):
    if response['answer'].strip() in sample['choice_list'][i]:
      sample['response']=i
      print("The answer is:",i)

  #print("The answer is:",sample['response'])

  0%|                                                   | 0/120 [00:00<?, ?it/s]

  1%|▎                                          | 1/120 [00:00<00:48,  2.47it/s]


Question:   
 Please pick the best option for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:In a small village, two farmers are working in their fields - a diligent farmer and a lazy farmer. The hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Can you explain this unusual relationship?
 
  
  Options:

  (A): The lazy farmer is his mother.

  (B): The lazy farmer is not a responsible father as he is lazy.

  (C): The diligent farmer devoted himself to the farm and gradually forgot his father.

  (D): None of above.

  
l'oggetto response è: (A): The lazy farmer is his mother.

  (B




  2%|▋                                          | 2/120 [00:00<00:56,  2.08it/s]


Question:   
 Please pick the best option for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:Romeo and Juliet are discovered dead on the bedroom floor. Glass shards and some water were on the floor when they were found. A bookcase and a bed are the sole pieces of furniture in the space. Other than the neighboring railroad track, the house is located in a rural area. How is that even doable?
 
  
  Options:

  (A): They were sleeping and scared by the sound of track.

  (B): The rumble of the train moved the shelf which 
crushed them.

  (C): Romeo and Juliet are fish. The rumble of the train knocked the tank off the shelf, it broke and Romeo and Juliet did not survive.

  (D): None of above.

  
l'oggetto response è: (A): They




...


Question:   
 Please pick the best option for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:There were five men at church, and it started raining while they were outside. The four that ran still got wet, but the one that was still stayed completely dry. Why did he stay dry?
 
  
  Options:

  (A): The man is an athlete that can avoid rain in high speed.

  (B): 
The man was lucky enough to avoid all the rain.

  (C): He was in a coffin the other men were at his funeral.


  (D): None of above.

  
l'oggetto response è: Options:

  (A): The man is an athlete that




 12%|████▉                                     | 14/120 [00:05<00:41,  2.55it/s]


Question:   
 Please pick the best option for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:On a Sunday afternoon, a woman spotted a bear in the distance. Rather than be afraid, she ran toward it excited. Why wasn't she afraid?
 
  
  Options:

  (A): She was at the zoo.


  (B): She become brave in sudden.

  (C): She tried to scare the bear.

  (D): None of above.

  
l'oggetto response è: (A): She was at the zoo.


  (B




 12%|████▉                                     | 14/120 [00:05<00:43,  2.41it/s]


KeyboardInterrupt: 

# Strip and final result

In [ ]:
for index,item in enumerate(test_data_list):
    try:
        item['predict'] = item['response'].split('(')[1].strip().split(')')[0].strip()
    except:
        print(index)

In [ ]:
word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play,sentence_play)